# Notion MCP Agent
## ABB #6 - Session 4

Code authored by: Shaw Talebi

Resources
- [YouTube video](https://youtu.be/lS33W56-NGc?si=ssxso76-f56lJbk-)

### imports

In [1]:
from agents.mcp.server import MCPServerStdio, MCPServer
from agents import Agent, Runner, gen_trace_id, trace, ModelSettings
from openai.types.responses import ResponseTextDeltaEvent
import os
import asyncio

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import logging
# Suppress httpx INFO logs to reduce console output
logging.getLogger("httpx").setLevel(logging.WARNING)

In [3]:
# import notion api key
notion_key = os.getenv("NOTION_API_KEY")

Create your API key here: https://www.notion.so/profile/integrations

### explore MCP server tools

In [4]:
async with MCPServerStdio(
    params={
        "command": "npx",
        "args": ["-y", "@notionhq/notion-mcp-server"],
        "env": {
        "OPENAPI_MCP_HEADERS": "{\"Authorization\": \"Bearer " + notion_key + "\", \"Notion-Version\": \"2022-06-28\" }"
        }
    }
) as server:
    tool_list = await server.list_tools()

In [5]:
toolname_list = [tool.name for tool in tool_list]
print(toolname_list)

['API-get-user', 'API-get-users', 'API-get-self', 'API-post-database-query', 'API-post-search', 'API-get-block-children', 'API-patch-block-children', 'API-retrieve-a-block', 'API-update-a-block', 'API-delete-a-block', 'API-retrieve-a-page', 'API-patch-page', 'API-post-page', 'API-create-a-database', 'API-update-a-database', 'API-retrieve-a-database', 'API-retrieve-a-page-property', 'API-retrieve-a-comment', 'API-create-a-comment']


In [6]:
[tool.description for tool in tool_list]

['Notion | Retrieve a user',
 'Notion | List all users',
 "Notion | Retrieve your token's bot user",
 'Notion | Query a database',
 'Notion | Search by title',
 'Notion | Retrieve block children',
 'Notion | Append block children',
 'Notion | Retrieve a block',
 'Notion | Update a block',
 'Notion | Delete a block',
 'Notion | Retrieve a page',
 'Notion | Update page properties',
 'Notion | Create a page',
 'Notion | Create a database',
 'Notion | Update a database',
 'Notion | Retrieve a database',
 'Notion | Retrieve a page property item',
 'Notion | Retrieve comments',
 'Notion | Create comment']

### create main with MCP server

In [7]:
async def main():
    
    async with MCPServerStdio(
        params={
            "command": "npx",
            "args": ["-y", "@notionhq/notion-mcp-server"],
            "env": {
            "OPENAPI_MCP_HEADERS": "{\"Authorization\": \"Bearer " + notion_key + "\", \"Notion-Version\": \"2022-06-28\" }"
            }
        }
    ) as server:
        trace_id = gen_trace_id()
        with trace(workflow_name="Notion Agent Example", trace_id=trace_id):
            print(f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}\n")
            await run(server)

### create run() with agent

In [8]:
with open('context/example-2/instructions.md', 'r', encoding='utf-8') as file:
    instructions = file.read()

In [9]:
async def run(mcp_server: MCPServer):
    agent = Agent(
        name="Notion Agent",
        model="gpt-4.1-2025-04-14",
        instructions=instructions,
        mcp_servers=[mcp_server],
    )
    ModelSettings.tool_choice = "required"
    
    input_items = []

    print("=== Notion AI Agent ===")
    print("Type 'exit' to end the conversation")

    while True:
        # Get user input
        user_input = input("\nUser: ").strip()
        input_items.append({"content": user_input, "role": "user"})
        
        # Check for exit command
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("\nGoodbye!")
            break
            
        if not user_input:
            continue

        print("\nAgent: ", end="", flush=True)
        result = Runner.run_streamed(
            agent,
            input=input_items,
        )
        
        async for event in result.stream_events():
            # We'll ignore the raw responses event deltas for text
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                print(event.data.delta, end="", flush=True)
            elif event.type == "run_item_stream_event":
                if event.item.type == "tool_call_item":
                    print(f"\n-- Calling Tool: {event.item.raw_item.name}...")
                elif event.item.type == "tool_call_output_item":
                    print("-- Tool call completed.")
                elif event.item.type == "message_output_item":
                    input_items.append({"content": f"{event.item.raw_item.content[0].text}", "role": "assistant"})
                else:
                    pass  # Ignore other event types

        print("\n")  # Add a newline after each response

### run agent

In [10]:
await main()
# find me posts about entreprenuership

View trace: https://platform.openai.com/traces/trace?trace_id=trace_109a254cc79e4748886c8c194ea7ebdc

=== Notion AI Agent ===
Type 'exit' to end the conversation



User:  find me posts about entreprenuership



Agent: 
-- Calling Tool: API-post-search...
-- Tool call completed.
Here are several posts about entrepreneurship in Shaw Talebi’s Notion workspace:

1. **High highs and low lows of entrepreneurship**
   - Covers the emotional ups and downs faced as an entrepreneur.
   - [View in Notion](https://www.notion.so/High-highs-and-low-lows-of-entrepreneurship-2605f2e26be98006886ad66abb6dfc21)

2. **15 things I had to learn to become a (data) entrepreneur**
   - A list of important lessons learned on the journey to becoming a data entrepreneur.
   - [View in Notion](https://www.notion.so/15-things-I-had-to-learn-to-become-a-data-entrepreneur-1445f2e26be980e98a06c47cf45d6f75)

3. **AI Roadmap for Entrepreneurs**
   - Breaks down the process of incorporating AI into entrepreneurship.
   - [View in Notion](https://www.notion.so/AI-Roadmap-for-Entrepreneurs-1e25f2e26be980a88bccd7f45555d29f)

4. **10 (non-technical) skills I needed to learn to survive as a data entrepreneur**
   - Shares essential


User:  based on the full content of these posts can you write me another one?



Agent: 
-- Calling Tool: API-get-block-children...

-- Calling Tool: API-get-block-children...

-- Calling Tool: API-get-block-children...

-- Calling Tool: API-get-block-children...

-- Calling Tool: API-get-block-children...

-- Calling Tool: API-get-block-children...

-- Calling Tool: API-get-block-children...

-- Calling Tool: API-get-block-children...

-- Calling Tool: API-get-block-children...

-- Calling Tool: API-get-block-children...
-- Tool call completed.
-- Tool call completed.
-- Tool call completed.
-- Tool call completed.
-- Tool call completed.
-- Tool call completed.
-- Tool call completed.
-- Tool call completed.
-- Tool call completed.
-- Tool call completed.
Let’s go step-by-step as instructed, drawing from the existing posts about entrepreneurship and balancing both practical and mindset advice. 

---

### Step 1: Purpose and Audience

- **Purpose:** To educate and support aspiring entrepreneurs by sharing overlooked but essential steps—and mistakes—on the journey


User:  exit



Goodbye!


In [11]:
# # to run in a .py script use
# if __name__ == "__main__":
#     asyncio.run(main())